<a href="https://colab.research.google.com/github/hejnal/py-study-pandas/blob/main/notebooks/PyStudy_Group_13_Exercise_2_BigFrames_ipynb_%5BMAKE_A_COPY%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyStudy Group #13 - Exercise 2 - BigFrames
The purpose of this notebook is practice BigFrames API and compare it with a classic Pandas development

## Install Dependencies

In [ ]:
%%bash
pip install -q --upgrade bigframes

In [ ]:
# Load BigQuery Magic extension
%load_ext google.cloud.bigquery

## Project Setup

In [6]:
# Importing BigFrames
import bigframes.pandas as bpd
# Importing the matplotlib library
import matplotlib.pyplot as plt

PROJECT_ID = "whejna-py-study"  # @param {type:"string"}
REGION = "EU"  # @param {type:"string"}
SCENARIOS = {
    "SMALL_DATA": 1000000,
    "MEDIUM_DATA": 10000000,
    "BIG_DATA": 100000000,
}

# Set the project ID
bpd.options.bigquery.project = PROJECT_ID
bpd.close_session()

# Set BigQuery DataFrames options
# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bpd.options.bigquery.project = PROJECT_ID

# Note: The location option is not required.
# It defaults to the location of the first table or query
# passed to read_gbq(). For APIs where a location can't be
# auto-detected, the location defaults to the "US" location.
bpd.options.bigquery.location = REGION

## Authenticate the notebook (Colab only)

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Util Functions

In [7]:
# Create some utils methods
from pandas import DataFrame

def calculate_hourly_hires(df: DataFrame):
  df['start_hour'] = df['start_date'].dt.hour
  hourly_hires = df.groupby('start_hour').size()
  return hourly_hires

def visualize_hourly_hires(df: DataFrame):
  plt.bar(df.index, df.values)
  plt.xlabel('Hour of the Day')
  plt.ylabel('Number of Hires')
  plt.title('Number of Bicycle Hires by Hour of the Day')

def visualize_elapsed_times(elapsed_times: dict):
  labels = list(elapsed_times.keys())
  values = list(elapsed_times.values())
  colors = ['blue', 'green', 'red']

  plt.bar(labels, values, color=colors)
  plt.xlabel("Methods")
  plt.ylabel("Elapsed Time")
  plt.title(f"Elapsed Time for Each Method, with the MAX_ROWS={MAX_ROWS}")
  plt.show()

# Use Case 1 [Pandas]: Benchmark London Bicycles Hire: Analyze peak usage hours

Using public dataset (located in the EU) Compare 3 different mechanisms to solve this problem:

1. Pandas DataFrame
2. BigQuery DataFrame
3. BigQuery Plan SQL

## Set Scenario (SMALL_DATA, MEDIUM_DATA or BIG_DATA)

In [8]:
MAX_ROWS = SCENARIOS["SMALL_DATA"]

In [9]:
# Some extra structures for measuring the elapsed times
elapsed_times = {
    "pandas": 0.0,
    "bigframes": 0.0,
    "direct_bq_sql": 0.0,
}

params = {
  "max_rows": MAX_ROWS
}

## Method 1: Pandas DataFrame

In [ ]:
%%timeit -n1 -r1 -o
%%bigquery cycle_hire_df --params $params --project $PROJECT_ID --no_query_cache
SELECT start_date, start_station_name, end_station_name, duration
FROM
`bigquery-public-data`.london_bicycles.cycle_hire
WHERE duration > 0
LIMIT @max_rows

In [19]:
# capture the time for the last operation
elapsed_times["pandas"] += _.best

In [ ]:
%%timeit -n1 -r1 -o
hourly_hires = calculate_hourly_hires(df=cycle_hire_df)
visualize_hourly_hires(df=hourly_hires)

In [21]:
# capture the time for the last operation
elapsed_times["pandas"] += _.best

## Method 2: BigQuery DataFrame

In [ ]:
import time # use time module, as the timeit runs in isolated scope and does not let to capture the output
start_time = time.time()
# Create a BigFrame from the cycle_hire table
cycle_hire_bf = bpd.read_gbq(
    "bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, columns=["start_date", "start_station_name", "end_station_name", "duration"],  filters=[("duration", ">", 0)]
)
end_time = time.time()
elapsed = end_time - start_time
elapsed_times["bigframes"] += elapsed

In [ ]:
%%timeit -n1 -r1 -o
hourly_hires_bf = calculate_hourly_hires(df=cycle_hire_bf)
visualize_hourly_hires(df=hourly_hires_bf)

In [ ]:
# capture the time for the last operation
elapsed_times["bigframes"] += _.best

## Method 3 Query in BigQuery

In [ ]:
%%timeit -n1 -r1 -o
%%bigquery peak_time --params $params --project $PROJECT_ID --no_query_cache
WITH base_with_limit AS (
  SELECT start_date
  FROM
    `bigquery-public-data`.london_bicycles.cycle_hire
  WHERE duration > 0
  LIMIT @max_rows
)
SELECT
  EXTRACT(HOUR FROM start_date) AS start_hour,
  COUNT(*) AS num_hires
FROM
    base_with_limit
GROUP BY 1
ORDER BY 1 ASC

In [ ]:
# capture the time for the last operation
elapsed_times["direct_bq_sql"] += _.best

In [ ]:
%%timeit -n1 -r1 -o
visualize_hourly_hires(df=peak_time["num_hires"])

In [ ]:
# capture the time for the last operation
elapsed_times["direct_bq_sql"] += _.best

## Compare the 3 different methods for a given batch size

In [ ]:
# SMALL DATA
visualize_elapsed_times(elapsed_times=elapsed_times)

## TODO: Exercise 1 Run Full Benchmark
Plot elapsed times for 2 other scenarios: MEDIUM_DATA and BIG_DATA

**Hint**: set MAX_ROWS to different scenarios at the beginning of the notebook and re-run the whole Scenario 1 block.

Scroll to the [Scenario 1 block](#scrollTo=v_H4pWawcQia) cell to set-up the new scenario.

In [ ]:
# Scenario: MEDIUM_DATA
visualize_elapsed_times(elapsed_times=elapsed_times)

In [ ]:
# Scenario: BIG_DATA
visualize_elapsed_times(elapsed_times=elapsed_times)

In [ ]:
# Convert to Pandas DataFrame
pd_df = average_duration_by_station.head(10).to_pandas()

# Plot using Pandas
plt.figure(figsize=(10, 6))
pd_df.plot(kind="bar")
plt.title("Top 10 Stations with Longest Average Rental Duration")
plt.xlabel("Station Name")
plt.ylabel("Average Duration (seconds)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Use Case 2 [Pandas]: Get Monthly Rental Trends and Most Popular Stations With BigFrames

In [ ]:
MAX_ROWS = SCENARIOS["SMALL_DATA"]

In [ ]:
# Create BigFrames from the tables (hires and stations)
cycle_hire_bfd = bpd.read_gbq("bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, filters=[("duration", ">", 0)])
cycle_stations_bfd = bpd.read_gbq("bigquery-public-data.london_bicycles.cycle_stations", use_cache=False)

## TODO: Exercise 2: Get Monthly Rental Trends

In [ ]:
# Calculate the average duration of rentals for each station
average_duration_by_station = # TODO: group by start_station_name and calculate mean of duration

# Display the top 10 stations with the longest average rental durations
print(average_duration_by_station.head(10))

# Analyze rental trends over time
# Extract the month from a date
cycle_hire_bfd["month"] = cycle_hire_bfd["start_date"].dt.month

# Group by month, and get the average duration
monthly_rentals = # TODO: group by month and get avg duration

# Convert the Series to a DataFrame for easier plotting
monthly_rentals = monthly_rentals.reset_index()

# Plotting
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.plot(
    monthly_rentals["month"],
    monthly_rentals["avg_duration"],
    marker="o",
    linestyle="-",
)
plt.xlabel("Month")
plt.ylabel("Average Rental Duration")
plt.title("Trend of Average Rental Duration Over Time")
plt.grid(True)
plt.show()

In [ ]:
# @title Solution

# Calculate the average duration of rentals for each station
average_duration_by_station = (
    cycle_hire_bfd.groupby("start_station_name")["duration"].mean().sort_values(ascending=False)
)

# Display the top 10 stations with the longest average rental durations
print(average_duration_by_station.head(10))

# Analyze rental trends over time
# Extract the month from a date
cycle_hire_bfd["month"] = cycle_hire_bfd["start_date"].dt.month

# Group by month, and get the average duration
monthly_rentals = cycle_hire_bfd.groupby(["month"]).agg(
    avg_duration=("duration", "mean")
)

# Convert the Series to a DataFrame for easier plotting
monthly_rentals = monthly_rentals.reset_index()

# Plotting
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.plot(
    monthly_rentals["month"],
    monthly_rentals["avg_duration"],
    marker="o",
    linestyle="-",
)
plt.xlabel("Month")
plt.ylabel("Average Rental Duration")
plt.title("Trend of Average Rental Duration Over Time")
plt.grid(True)
plt.show()

## TODO: Exercise 3: Plot on the map 100 most busy stations.

In [ ]:
!pip install geopandas shapely geodatasets matplotlib -q

In [ ]:
!wget -P /content/sample_data/ https://github.com/gicentre/data/blob/03ad5e41d3023c48c047bff9dffc574933c07a67/geoTutorials/London_Borough_Excluding_MHW.shp

For this exercise use pandas DF, it is much faster

In [ ]:
cycle_hire_df = cycle_hire_bf.to_pandas()
cycle_stations_df = cycle_stations_bf.to_pandas()

TODO: Fix the following aggregations

In [ ]:
joined_df = # TODO: Hint use merge operation between cycle_hire_df and cycle_stations_df

most_popular_stations = # TODO: first use filter (start_station_name, longitude, latitude), then group by the station name and then apply few aggregations (size, first of longitude and latitude)

top_n_stations = # TODO: get top 100 stations
bottom_n_stations = # TODO: get bottom 100 stations

In [ ]:
# @title Solution
joined_df = cycle_hire_df.merge(cycle_stations_df, left_on="start_station_name", right_on="name", how="inner")

most_popular_stations = joined_df.filter(items=["start_station_name", "longitude", "latitude"]).groupby("start_station_name").agg(
    count=("start_station_name", "size"), # Count occurrences
    longitude=("longitude", "first"), # Take first longitude
    latitude=("latitude", "first")  # Take first latitude
).sort_values(by="count", ascending=False)

top_n_stations = most_popular_stations.head(100)
bottom_n_stations = most_popular_stations.tail(100)

Define some helper methods

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyogrio import set_gdal_config_options

set_gdal_config_options({
    'SHAPE_RESTORE_SHX': 'YES',
  })

def visualize_station_geo(top_df: pd.DataFrame, bottom_df: pd.DataFrame):

  london = gpd.read_file('/content/sample_data/London_Borough_Excluding_MHW.shp')
  london = london.set_crs("epsg:27700")

  # Use the new syntax here
  london['geometry'] = london['geometry'].to_crs('epsg:4326')

  top_gdf = gpd.GeoDataFrame(top_df, crs = 'epsg:4326', geometry = gpd.points_from_xy(top_df['longitude'], top_df['latitude']))
  bottom_gdf = gpd.GeoDataFrame(bottom_df, crs = 'epsg:4326', geometry = gpd.points_from_xy(bottom_df['longitude'], bottom_df['latitude']))

  ax = london.plot(figsize=(30, 12))
  top_gdf.plot(ax=ax, marker='o', color='yellow', markersize=30)
  bottom_gdf.plot(ax=ax, marker='o', color='red', markersize=30)

  ax.set_xlim(london.total_bounds[0], london.total_bounds[2])
  ax.set_ylim(london.total_bounds[1], london.total_bounds[3])

  plt.show()

Visualize the stations on the London map.

In [ ]:
visualize_station_geo(top_df=top_n_stations, bottom_df=bottom_n_stations)

# Use Case 3 [Pandas]: Custom UDFs via Remote Functions

Enable APIs

In [ ]:
!gcloud --project $PROJECT_ID services enable bigqueryconnection.googleapis.com cloudfunctions.googleapis.com run.googleapis.com cloudbuild.googleapis.com artifactregistry.googleapis.com cloudresourcemanager.googleapis.com

Consult the help around remote functions in BigFrames

In [ ]:
help(bpd.remote_function)

In [ ]:
import bigframes.pandas as bpd

# Load the dataframe
cycle_hire_bf = bpd.read_gbq(
    "bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, columns=["duration"],  filters=[("duration", ">", 0)]
)

# Define a remote function to categorize rental duration
@bpd.remote_function(
    int,  # Input type (duration in seconds)
    str,  # Output type (category)
    reuse=False,
)
def categorize_duration(duration_sec: int) -> str:
    if duration_sec <= 300:  # 5 minutes
        return "Short Trip"
    elif duration_sec <= 1800:  # 30 minutes
        return "Medium Trip"
    else:
        return "Long Trip"

# Apply the remote function to create a new column
cycle_hire_bf = cycle_hire_bf.assign(
    trip_category=cycle_hire_bf["duration"].apply(categorize_duration)
)

# Display the results
print(cycle_hire_bf[["duration", "trip_category"]].head())

# Use Case 4 [ML]: Predict Rental Duration with BigFrames ML and Scikit Learn

Load weather data from public dataset (load to pandas, as they are in a different region). Only use data for 2022 and a single London Weather Station (HEATHROW)

In [ ]:
%%bigquery weather_data_df --project $PROJECT_ID
SELECT
  date, value as precipitation
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2022` AS wx
WHERE
  date >= "2022-01-01"
  AND date <= "2022-12-31"
  AND id = 'UKM00003772'
  AND qflag IS NULL
  AND element = 'PRCP'
ORDER BY
  wx.date

Prepare the features for ML modelling

In [ ]:
MAX_ROWS = SCENARIOS["MEDIUM_DATA"]    # Load full dataset for ML

# 1. Load the bike rentals data from BigQuery and prepare the features
# Load the dataframe
cycle_hire_bdf = bpd.read_gbq(
    "bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, columns=["start_date", "start_station_name", "duration"],  filters=[("duration", ">", 0)]
)

cycle_hire_df = cycle_hire_bdf.to_pandas()   # cannot join between US and EU, so use both dataframes in pandas, pay attention to the LOAD jobs

cycle_hire_df['date'] = cycle_hire_df['start_date'].dt.date
cycle_hire_df['hour'] = cycle_hire_df['start_date'].dt.hour
cycle_hire_df['dayofweek'] = cycle_hire_df['start_date'].dt.dayofweek
cycle_hire_df['month'] = cycle_hire_df['start_date'].dt.month
cycle_hire_df['year'] = cycle_hire_df['start_date'].dt.year

filtered_df = cycle_hire_df.loc[:, ["year", "date", "dayofweek", "start_station_name"]].loc[cycle_hire_df["year"] == 2022]
num_rentals_df = filtered_df.groupby(["date", "dayofweek", "start_station_name"]).size().reset_index(name="num_rentals")

# join with the weather data
ready_for_ml = num_rentals_df.merge(weather_data_df, left_on="date", right_on="date", how="left")

# 2. Preprocess the data
ready_for_ml = ready_for_ml.dropna()

## Classic Scikit learn model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

training_data_pandas = ready_for_ml

# Pick feature columns and label column
# define features
X = training_data_pandas[
    [
        "dayofweek",
        "start_station_name",
        "precipitation"
    ]
]
# define target
y = training_data_pandas["num_rentals"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create preprocessing transformers
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, ['precipitation']), ('cat', categorical_transformer, ['dayofweek', 'start_station_name'])
])

from sklearn.linear_model import LinearRegression

# Create and train the model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression(fit_intercept=False))])

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
import matplotlib.pyplot as plt

# Get the minimum and maximum values from both y_test and y_pred
min_val = min(min(y_test), min(y_pred))
max_val = max(max(y_test), max(y_pred))

# Create the scatter plot
plt.scatter(y_test, y_pred)

# Set the x and y axis limits to be the same
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)

# Add labels and title
plt.xlabel("Actual Values (y_test)")
plt.ylabel("Predicted Values (y_pred)")
plt.title("Actual vs. Predicted Values")

# Add a diagonal line for reference (optional)
plt.plot([min_val, max_val], [min_val, max_val], 'k--')  # Dashed diagonal line

# Show the plot
plt.show()

## Equivalent ML Model with BigFrames ML (BQML)

In [ ]:
import bigframes.pandas as bpd
from bigframes.ml.linear_model import LinearRegression

# Split data into features (X) and target variable (y)
# define features
X = ready_for_ml[
    [
        "dayofweek",
        "start_station_name",
        "precipitation"
    ]
]
# define target
y = ready_for_ml["num_rentals"]

# Create and train the Linear Regression model
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

# 5. Evaluate the model (optional)
print(f"R-squared: {model.score(X, y)}")

Predict on a no rain day:

In [ ]:
import datetime

# 6. Make predictions (example)
# Create a new DataFrame with the same features for prediction
no_rain_monday_day = bpd.DataFrame(
    {
        "dayofweek": 0,
        "start_station_name": "Black Lion Gate, Kensington Gardens",
        "precipitation": 0
    },
    index=[0]
)

predictions = model.predict(no_rain_monday_day)
predictions.head(1)

Predict on a heavy rain day.

In [ ]:
import datetime

# 6. Make predictions (example)
# Create a new DataFrame with the same features for prediction
heavy_rain_monday_day = bpd.DataFrame(
    {
        "dayofweek": 0,
        "start_station_name": "Black Lion Gate, Kensington Gardens",
        "precipitation": 1000
    },
    index=[0]
)

predictions = model.predict(heavy_rain_monday_day)
predictions.head(1)

## TODO: Exercise 4: Improve the Model
 Improve the model by improving the features or switching to the more powerful regressor, check documentation

 https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.ensemble

 Hint: add isWeekend
 Hint: use XGBRegressor

In [ ]:
import bigframes.pandas as bpd
from bigframes.ml.linear_model import LinearRegression
# TODO: import XGBRegressor regressor

# 1. Load the bike rentals data from BigQuery and prepare the features
# Load the dataframe
cycle_hire_bdf = bpd.read_gbq(
    "bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, columns=["start_date", "start_station_name", "duration"],  filters=[("duration", ">", 0)]
)
cycle_hire_df = cycle_hire_bdf.to_pandas()   # cannot join between US and EU, so use both dataframes in pandas, pay attention to the LOAD jobs
cycle_hire_df['date'] = cycle_hire_df['start_date'].dt.date
cycle_hire_df['hour'] = cycle_hire_df['start_date'].dt.hour
cycle_hire_df['dayofweek'] = cycle_hire_df['start_date'].dt.dayofweek
cycle_hire_df['month'] = cycle_hire_df['start_date'].dt.month
cycle_hire_df['year'] = cycle_hire_df['start_date'].dt.year
# TODO: add more features

filtered_df = cycle_hire_df.loc[:, ["year", "date", "dayofweek", "start_station_name"]].loc[cycle_hire_df["year"] == 2022]
num_rentals_df = filtered_df.groupby(["date", "dayofweek", "start_station_name"]).size().reset_index(name="num_rentals")

# join with the weather data
ready_for_ml = num_rentals_df.merge(weather_data_df, left_on="date", right_on="date", how="left")

# 2. Preprocess the data
ready_for_ml = ready_for_ml.dropna()

# 3. Split data into features (X) and target variable (y)
# define features
X = ready_for_ml[
    [
        "dayofweek",
        "start_station_name",
        "precipitation"
    ]
]
# define target
y = ready_for_ml["num_rentals"]

# 4. Try different regressors
regressors = {
    "Linear Regression": LinearRegression(fit_intercept=False),
    # TODO: use XGBRegressor regressor
}

for name, regressor in regressors.items():
    # Create and train the model
    model = Pipeline(steps=[('regressor', regressor)])
    model.fit(X_train, y_train)

    # 5. Evaluate the model
    print(f"{name}: R-squared: {model.score(X_test, y_test)}")

In [ ]:
# @title Solution
import bigframes.pandas as bpd
from bigframes.ml.linear_model import LinearRegression
from bigframes.ml.ensemble import XGBRegressor

# 1. Load the bike rentals data from BigQuery and prepare the features
# Load the dataframe
cycle_hire_bdf = bpd.read_gbq(
    "bigquery-public-data.london_bicycles.cycle_hire", max_results=MAX_ROWS, use_cache=False, columns=["start_date", "start_station_name", "duration"],  filters=[("duration", ">", 0)]
)
cycle_hire_df = cycle_hire_bdf.to_pandas()   # cannot join between US and EU, so use both dataframes in pandas, pay attention to the LOAD jobs
cycle_hire_df['date'] = cycle_hire_df['start_date'].dt.date
cycle_hire_df['hour'] = cycle_hire_df['start_date'].dt.hour
cycle_hire_df['dayofweek'] = cycle_hire_df['start_date'].dt.dayofweek
cycle_hire_df['isweekend'] = cycle_hire_df['start_date'].dt.dayofweek >= 5
cycle_hire_df['month'] = cycle_hire_df['start_date'].dt.month
cycle_hire_df['year'] = cycle_hire_df['start_date'].dt.year

filtered_df = cycle_hire_df.loc[:, ["year", "date", "dayofweek", "isweekend", "start_station_name"]].loc[cycle_hire_df["year"] == 2022]
num_rentals_df = filtered_df.groupby(["date", "dayofweek", "isweekend", "start_station_name"]).size().reset_index(name="num_rentals")

# join with the weather data
ready_for_ml = num_rentals_df.merge(weather_data_df, left_on="date", right_on="date", how="left")

# 2. Preprocess the data
ready_for_ml = ready_for_ml.dropna()

# 3. Split data into features (X) and target variable (y)
# define features
X = ready_for_ml[
    [
        "dayofweek",
        "isweekend",
        "start_station_name",
        "precipitation"
    ]
]
# define target
y = ready_for_ml["num_rentals"]

# 4. Try different regressors
regressors = {
    "Linear Regression": LinearRegression(fit_intercept=False),
    "XGB Regressor": XGBRegressor()
}

for name, regressor in regressors.items():
    # Create and train the model
    model = Pipeline(steps=[('regressor', regressor)])
    model.fit(X_train, y_train)

    # 5. Evaluate the model
    print(f"{name}: R-squared: {model.score(X_test, y_test)}")